# To-do's
* Clustering

In [ ]:
# Models
from keras.models import Sequential
from keras.layers import Dense, MaxPooling2D, Flatten, Conv2D
from keras.wrappers.scikit_learn import KerasClassifier

# Preprocessing
from sklearn.preprocessing import normalize
from keras.preprocessing.image import ImageDataGenerator

# Validation
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

# Visualization
from sklearn.manifold import LocallyLinearEmbedding, SpectralEmbedding
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='white', context='notebook', palette='deep')

# Core
import pandas as pd
import numpy as np

In [ ]:
ts = pd.read_csv('../input/train.csv')
vd = pd.read_csv('../input/test.csv')

In [ ]:
df = pd.concat([ts, vd], ignore_index=True)
df.columns = map(lambda x: "{:03d}".format(int('0'+x[5:])) if x!='label' else x, df.columns)
df = df[sorted(df.columns)]

In [ ]:
num_classes = 10
X, y = df.loc[:,df.columns != 'label'], df.loc[:, 'label']

# Exploration

## Clustering

In [ ]:
#cluster = SpectralEmbedding(n_components=2)
#z = cluster.fit_transform(X.loc[np.isnan(y), :]).T

In [ ]:
#clusters = [LocallyLinearEmbedding(n_components=2), SpectralEmbedding(n_components=2)]
#fig, axs = plt.subplots(len(clusters), 1, figsize=(20,30))
#for i, cluster in enumerate(clusters):
#    z = cluster.fit_transform(X.loc[np.isnan(y), :]).T
#    axs[i] = plt.scatter(z[0], z[1], c=ytrain)

In [ ]:
number_n = 10
f, axs = plt.subplots(1, number_n, figsize=(20,3))

print("Examples of each digit")
for n in range(number_n-1, -1, -1):
    ax = axs[n]
    sample = y[y==n].sample(1).index
    ax.imshow(X.iloc[sample].values.reshape(28, 28))
    ax.set_title(u"Dígito {:d}".format(int(y[sample])))
    ax.set_ylabel('')
    ax.set_xlabel('')
    ax.set_yticklabels([])
    ax.set_xticklabels([])
plt.show();

## Counts Each Digit

In [ ]:
fig, ax = plt.subplots()
sns.countplot(y[y.notna()].apply(int))
ax.set_ylabel("Qty.")
ax.set_xlabel("Digit")
plt.show();

# Preprocessing

## Normalization

In [ ]:
fig, ax = plt.subplots(1, 2)
sample = y[y.notna()].sample(1).index
ax[0].imshow(X.iloc[sample].values.reshape(28, 28))
X_norm = normalize(X, axis=0, norm='max');
ax[1].imshow(X_norm[sample].reshape(28, 28))

In [ ]:
X = X_norm.reshape(X.shape[0], 28, 28, 1)

# Modeling

In [ ]:
train = y.notna()
test = ~train

# convert to binary matrix
y_matrix = (y[:,None] == range(num_classes)).astype(int)

Xtrain, ytrain = X[train], y_matrix[train]
Xtest , ytest  = X[test] , y_matrix[test]

In [ ]:
def baseline_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(6, 6), strides=(2, 2), activation='relu',input_shape=Xtrain[0].shape))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Conv2D(64, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(1000, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
    return model

In [ ]:
#estimator = KerasClassifier(build_fn=baseline_model, epochs=10, batch_size=32, verbose=1)
estimator = baseline_model()

In [ ]:
cv_test = False
if cv_test:
    kfold = KFold(n_splits=3, shuffle=True, random_state=43)
    results = cross_val_score(estimator, Xtrain, ytrain, cv=kfold)
    print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

## Augmentation

In [ ]:
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=None,
    shear_range=0.1,
    zoom_range=0.1,
    channel_shift_range=0.0,
    fill_mode='nearest',
    cval=0.0,
    horizontal_flip=False,
    vertical_flip=False,
    rescale=None,
    preprocessing_function=None,
    data_format=None,
    validation_split=0.2
)

In [ ]:
datagen.fit(Xtrain)

In [ ]:
Xtrain.shape

In [ ]:
history = estimator.fit_generator(
    datagen.flow(Xtrain, ytrain, batch_size=32),
    steps_per_epoch=Xtrain.shape[0],
    epochs=10
)

In [ ]:
plt.figure(figsize=(10,5))
plt.scatter(range(1, len(history.history['loss'])+1), history.history['loss'], label='loss')
plt.scatter(range(1, len(history.history['acc'])+1), history.history['acc'], label='acc')
plt.legend()
plt.show();

In [ ]:
ytest = estimator.predict_classes(Xtest)

In [ ]:
submit = pd.DataFrame(data={'ImageId': range(1, ytest.shape[0]+1), 'Label': ytest})

In [ ]:
submit.to_csv("submit.csv", index=None)